In [1]:
import pandas as pd
import requests
import json
import os

## Select unique genes from HGMD

In [2]:
csvpath = './data/goflof_HGMD2019_v032021_allfeat.csv'

In [3]:
goflof = pd.read_csv(csvpath)
goflof

,ID,LABEL,CHROM,POS,GENE,RefSeq,HGVSc,HGVSp,MAX_AF,cDNA_position,...,coil_prob,Pfam_dom,DOMAINS_VEP,Clarks_distance,PTM,Phosphorylation,Acetylation,Methylation,Ubiquitination,Glycosylation
0,CD010589,LOF,12,53708608,AAAS,NM_015665.5,ENST00000209873.4:c.470_471del,ENSP00000209873.4:p.Phe157CysfsTer16,NaN,636-637/1840,...,0.040,Outside_domain,"Superfamily_domains:SSF82171,SMART_domains:SM0...",1.05664,0,0,0,0,0,0
1,CD010590,LOF,12,53701655,AAAS,NM_015665.5,ENST00000209873.4:c.1389del,ENSP00000209873.4:p.Phe464SerfsTer87,NaN,1555/1840,...,0.979,Outside_domain,"Superfamily_domains:SSF82171,hmmpanther:PTHR14...",1.05664,0,0,0,0,0,0
2,CM010147,LOF,12,53715207,AAAS,NM_015665.5,ENST00000209873.4:c.43C>A,ENSP00000209873.4:p.Gln15Lys,0.000552,209/1840,...,0.381,Outside_domain,"hmmpanther:PTHR14494,hmmpanther:PTHR14494:SF0",1.05664,0,0,0,0,0,0
3,CM010148,LOF,12,53714349,AAAS,NM_015665.5,ENST00000209873.4:c.251G>A,ENSP00000209873.4:p.Trp84Ter,0.000089,417/1840,...,0.279,Outside_domain,"hmmpanther:PTHR14494,hmmpanther:PTHR14494:SF0",1.05664,0,0,0,0,0,0
4,CM010149,LOF,12,53708601,AAAS,NM_015665.5,ENST00000209873.4:c.479A>G,ENSP00000209873.4:p.His160Arg,NaN,645/1840,...,0.858,Outside_domain,"Superfamily_domains:SSF82171,SMART_domains:SM0...",1.05664,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9614,CM1411787,LOF,10,298399,ZMYND11,NM_006624.5,ENST00000397962.3:c.1798C>T,ENSP00000381053.3:p.Arg600Trp,NaN,2226/4393,...,0.638,Outside_domain,"hmmpanther:PTHR24102,Superfamily_domains:SSF14...",0.85179,0,0,0,0,0,0
9615,CM149069,LOF,10,292731,ZMYND11,NM_006624.5,ENST00000397962.3:c.976C>T,ENSP00000381053.3:p.Gln326Ter,NaN,1404/4393,...,0.773,PF00855,"Gene3D:2.30.30.160,Pfam_domain:PF00855,PROSITE...",0.85179,0,0,0,0,0,0
9616,CM1512351,LOF,16,49669693,ZNF423,NM_015069.4,ENST00000561648.1:c.3370G>A,ENSP00000455426.1:p.Glu1124Lys,0.001000,3424/7660,...,0.800,PF00096,"PROSITE_profiles:PS50157,hmmpanther:PTHR24385:...",0.97752,0,0,0,0,0,0
9617,CM1512352,LOF,16,49764693,ZNF423,NM_015069.4,ENST00000561648.1:c.266G>A,ENSP00000455426.1:p.Arg89His,0.000580,320/7660,...,0.903,PF13912,"hmmpanther:PTHR24385:SF49,hmmpanther:PTHR24385...",0.97752,0,0,0,0,0,0


In [4]:
goflof['Consequence'].value_counts()

missense_variant                                                  5436
frameshift_variant                                                1858
stop_gained                                                       1502
missense_variant,splice_region_variant                             317
inframe_deletion                                                   175
frameshift_variant,splice_region_variant                            63
start_lost                                                          58
stop_gained,splice_region_variant                                   57
inframe_insertion                                                   29
stop_gained,frameshift_variant                                      25
splice_region_variant,synonymous_variant                            20
synonymous_variant                                                  16
stop_lost                                                           15
splice_donor_variant,coding_sequence_variant                        11
splice

In [10]:
genes = goflof['GENE'].unique().tolist()

## Requesting gnomAD API 

In [6]:
query = '''
{
  gene(gene_symbol: "%s", reference_genome: GRCh38) {
    gene_id
    start
    stop
    chrom
    variants(dataset: gnomad_r3) {
      variant_id
      pos
      ref
      alt
      reference_genome
      transcript_consequence {
        major_consequence
        polyphen_prediction
      }
    }
  }
}
'''

In [7]:
end_point = "https://gnomad.broadinstitute.org/api/"

In [8]:
import time

def getdata(gene):
    """
    Returns the response from gnomad API for a gene as well as the attempt number. If 429 errors, then retries else 
    returns -1, {}.
    If all retries fail, returns, -2, {}.
    """
    max_retries = 10
    initial_delay = 1
    backoff_factor = 2
    
    
    
    for attempt in range(max_retries):
        response = requests.post(end_point, data={'query': query % (gene)}, timeout=None)
        print(f'attempt: {attempt + 1}')
        if response.status_code == 200:
            if 'errors' not in response.json():
                return response.json()['data']
            else:
                return {'error': response.json()['errors']}
        elif response.status_code == 429:
            delay = initial_delay * (backoff_factor ** (attempt + 1))
            if attempt < max_retries - 1:
                time.sleep(delay)   
        else:
            return {'error': response.status_code}
    return {'error': -2}

In [ ]:
# genes.append('GALT')
gnomad_data = []
bad = []
from IPython.display import clear_output
while genes:
    gene = genes.pop(0)
    print(len(genes))
    print(gene)
    gene_data = getdata(gene)
    if 'error' in gene_data:
        if isinstance(gene_data['error'], list):
            bad.append(gene)
        else:
            genes.append(gene)
    else:
        gnomad_data.append(gene_data)
    clear_output()
    time.sleep(1)

1324
ATP13A2
attempt: 1


In [ ]:
print(bad)
print(len(gnomad_data))

## Store results

In [ ]:
if not os.path.exists('./output'):
    os.mkdir('./output')

In [ ]:
import json

jsonpath = './output/gnomad_data.json'

with open(jsonpath, 'w') as file:
    json.dump(gnomad_data, file)